<a href="https://colab.research.google.com/github/MarcoTheLoco/BTC-gold-levels/blob/main/BTC_gold_levels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Этот код выполняет сравнительный анализ эффективности двух типов ценовых уровней биткоина: фиксированных в долларах (***USD***) и фиксированных в граммах золота (***Gold***).

Код проходит через 4 основных этапа:

*Сбор данных*: Скачивает историю цен BTC и Золота с 2017 года.

*Генерация уровней*: Берет 7 исторических точек (экстремумов).
Для каждой точки запоминает цену в USD (например, $19,114) и вес в Золоте (например, 475g).

**USD уровень** — это прямая линия (цена не меняется).

**Gold уровень** — это кривая линия (наш заданный вес 475g умножается на текущую цену золота каждый день).

Если цена пробивает эту зону (пересекает её насквозь) — это считается "Пересечением" (Cross). Это плохо, это признак нестабильности уровня.

Если цена актива находится в диапазоне среднего шума от уровня — это считается "Удержанием" (Respect).

Компьютер не может «видеть» график глазами, поэтому мы используем логику состояний.

В коде это реализовано через переменную state

Она может принимать три значения:

*1 (Цена над уровнем)*

*-1 (Цена под уровнем)*

*0 (Цена внутри уровня, в зоне шума) — начальное состояние*

Как алгоритм понимает, что случилось пересечение?
Алгоритм идет по графику день за днем и на каждом шаге проверяет: «Где я нахожусь относительно уровня?»

Логика пересечения (Crossover) срабатывает ТОЛЬКО в момент, когда цена полностью меняет сторону, на которой она находится.

Вот пошаговый пример того, как думает код:

**День 1 (Цена высоко):**

Дистанция +10% (это больше шума).

Код запоминает: state = 1 (Мы СВЕРХУ).

**День 2 (Цена упала, вошла в зону):**

Дистанция +2% (это внутри зоны шума).

Код говорит: "О, это Удержание (Respect)!". Рисует зеленую точку.

Но state остается 1. Мы все еще считаем, что мы пришли сверху. Мы еще не пробили уровень, мы просто "трогаем" его.

**День 3 (Цена еще упала, но все еще в зоне):**

Дистанция -2% (все еще внутри зоны).

Код говорит: "Снова Удержание!". Рисует зеленую точку.

state все еще 1. Мы пока не считаем это пробоем.

**День 4 (Цена провалилась глубоко вниз):**

Дистанция -8% (это ниже шума).

Код видит: "Текущая позиция СНИЗУ (-1)".

А прошлая запомненная позиция была СВЕРХУ (1).

СОБЫТИЕ! 1 сменилось на -1.

**ВЫВОД:** "Произошло Пересечение (Crossover)!". Рисуем красный крестик.

Обновляем память: теперь state = -1 (мы официально внизу).

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats
import warnings
import yfinance as yf
import os

warnings.filterwarnings('ignore')

# тут я выбрал датасет, с которого начинаются текущие уровни, которые учитываются в прогнозировании по сей день в крипто-кругах
# а именно 2017й год, где биткоин и все альткоины обновили свои ATH значения
print("Loading dataset")
btc = yf.download("BTC-USD", start="2017-01-01", end="2025-12-06", progress=False)
gold = yf.download("GC=F", start="2017-01-01", end="2025-12-06", progress=False)

df = pd.DataFrame()
if isinstance(btc.columns, pd.MultiIndex):
    df['BTC_USD'] = btc[('Close', 'BTC-USD')]
else:
    df['BTC_USD'] = btc['Close']

if isinstance(gold.columns, pd.MultiIndex):
    df['XAU_USD'] = gold[('Close', 'GC=F')]
else:
    df['XAU_USD'] = gold['Close']

df['XAU_USD'] = df['XAU_USD'].ffill()
df = df.dropna()
print(f"Данные готовы: {len(df)} записей.\n")

# верхние экстремумы от которых мы строим значения

levels_config = [
    {'date': '2017-06-11', 'gold_grams': 73,   'name': 'L1 (73g)'},
    {'date': '2017-12-18', 'gold_grams': 475,  'name': 'L2 (475g)'},
    {'date': '2019-06-26', 'gold_grams': 287,  'name': 'L3 (287g)'},
    {'date': '2021-01-09', 'gold_grams': 700,  'name': 'L4 (700g)'},
    {'date': '2021-02-22', 'gold_grams': 995,  'name': 'L6 (995g)'},
    {'date': '2021-03-13', 'gold_grams': 1100, 'name': 'L7 (1100g)'},
    {'date': '2021-04-14', 'gold_grams': 1110, 'name': 'L5 (1110g)'},
]

Loading dataset
Данные готовы: 3259 записей.



*Параметр* допустимого отклонения (Tolerance Zone) был установлен
на уровне 5% ($ \alpha = 0.05 $).

Данный выбор обусловлен высокой исторической волатильностью актива: средний внутридневной размах цен (**High/Low Range**) Биткоина за исследуемый период (2017–2025) составляет 4.75%. Использование более узкого диапазона (<4%) привело бы к появлению ошибок первого рода (ложноположительные пробои), когда стандартный рыночный шум интерпретируется как нарушение структурного уровня.

In [12]:
# Рассчитаем волатильность на основе уже загруженного df
# Внутридневной размах в % = (High - Low) / Low

btc_full = yf.download("BTC-USD", start="2017-01-01", end="2025-12-06", progress=False)

if isinstance(btc_full.columns, pd.MultiIndex):
    high = btc_full[('High', 'BTC-USD')]
    low = btc_full[('Low', 'BTC-USD')]
else:
    high = btc_full['High']
    low = btc_full['Low']

daily_volatility = (high - low) / low

print(f"Средний внутридневной размах: {daily_volatility.mean():.2%}")
print(f"Медианный размах:             {daily_volatility.median():.2%}")
print(f"Максимальный размах (крах):   {daily_volatility.max():.2%}")
print("-" * 40)

if ZONE_PCT > daily_volatility.mean():
    print(f"Порог ({ZONE_PCT:.1%}) выше среднего шума ({daily_volatility.mean():.10%}).")
else:
    print(f"Порог ниже уровня шума.")


Средний внутридневной размах: 4.75%
Медианный размах:             3.70%
Максимальный размах (крах):   63.14%
----------------------------------------
Порог (5.0%) выше среднего шума (4.7456670242%).


In [15]:
ZONE_PCT = 0.05 #наша граница фильтрации шума, которую мы достали выше


def analyze_immortal(df, level_series):
    dist = (df['BTC_USD'] - level_series) / level_series
    respect_points = []
    cross_points = []
    state = 0
    if len(dist) > 0:
        if dist.iloc[0] > ZONE_PCT: state = 1
        elif dist.iloc[0] < -ZONE_PCT: state = -1

    for i in range(1, len(df)):
        curr_dist = dist.iloc[i]
        if abs(curr_dist) <= ZONE_PCT:
            respect_points.append((df.index[i], df['BTC_USD'].iloc[i]))
        new_state = state
        if curr_dist > ZONE_PCT: new_state = 1
        elif curr_dist < -ZONE_PCT: new_state = -1
        if new_state != 0 and state != 0 and new_state != state:
            cross_points.append((df.index[i], df['BTC_USD'].iloc[i]))
            state = new_state
        elif state == 0 and new_state != 0:
            state = new_state
    return respect_points, cross_points


stats_list = []
saved_files = []

for i, conf in enumerate(levels_config):
    d_start = pd.to_datetime(conf['date'])
    if d_start not in df.index:
        idx = df.index.searchsorted(d_start)
        if idx < len(df): d_start = df.index[idx]
        else: continue

    sub_df = df.loc[d_start:].copy()
    if sub_df.empty: continue

    # USD
    price_start = df['BTC_USD'].asof(d_start)
    u_ser = pd.Series(price_start, index=sub_df.index)
    u_resp, u_cross = analyze_immortal(sub_df, u_ser)

    # Gold
    g_oz = conf['gold_grams'] / 31.1035
    g_ser = sub_df['XAU_USD'] * g_oz
    g_resp, g_cross = analyze_immortal(sub_df, g_ser)

    # Сбор статистики
    stats_list.append({
        'Name': conf['name'],
        'Gold_Respect': len(g_resp),
        'USD_Respect': len(u_resp),
        'Gold_Cross': len(g_cross),
        'USD_Cross': len(u_cross),
        'Date': conf['date']
    })

    # Сохранение картинок
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    fig.patch.set_facecolor('#1a1a2e')
    # GOLD
    ax1.set_facecolor('#1a1a2e')
    ax1.plot(sub_df.index, sub_df['BTC_USD'], color='white', alpha=0.5, linewidth=0.8)
    ax1.plot(sub_df.index, g_ser, color='magenta', linewidth=2, label=f"Gold {conf['gold_grams']}g")
    if g_resp: ax1.scatter(*zip(*g_resp), color='#00ff00', s=10, alpha=0.6)
    if g_cross: ax1.scatter(*zip(*g_cross), color='red', s=100, marker='x', linewidth=2)
    ax1.set_yscale('log'); ax1.tick_params(colors='white'); ax1.set_title(f"GOLD: {conf['name']}", color='white')
    # USD
    ax2.set_facecolor('#1a1a2e')
    ax2.plot(sub_df.index, sub_df['BTC_USD'], color='white', alpha=0.5, linewidth=0.8)
    ax2.axhline(y=price_start, color='yellow', linewidth=2, linestyle='--', label="USD Level")
    if u_resp: ax2.scatter(*zip(*u_resp), color='#00ff00', s=10, alpha=0.6)
    if u_cross: ax2.scatter(*zip(*u_cross), color='red', s=100, marker='x', linewidth=2)
    ax2.set_yscale('log'); ax2.tick_params(colors='white'); ax2.set_title(f"USD: {conf['name']}", color='white')

    fname = f"analysis_{i+1}_{conf['gold_grams']}g.png"
    plt.tight_layout()
    plt.savefig(fname, dpi=100, facecolor='#1a1a2e')
    plt.close(fig)
    saved_files.append(fname)

print(f"Графики сохранены ({len(saved_files)} шт).")


Графики сохранены (7 шт).


Далее мы собираем эконометрическую статистику и и проводим парный T-тест (Paired T-test).

Чтобы сравнить эффективность, мы вводим синтетическую метрику Quality Score (Качество уровня).

В коде это строки:

gold_scores = [x['Gold_Respect'] - (x['Gold_Cross']*5) for x in stats_list]

usd_scores = [x['USD_Respect'] - (x['USD_Cross']*5) for x in stats_list]

Логика: Мы награждаем уровень за каждый день удержания цены (+1) и штрафуем за каждый пробой (-5).

Почему так? Потому что хороший уровень должен долго держать цену и редко пробиваться. Если уровень "решето" (много пробоев), его рейтинг падает.


*Наша нулевая гипотеза* ***H0:***

Суть: "Разницы между Золотыми и Долларовыми уровнями нет. Любые отличия в результатах — это просто случайность".

Математически: Среднее значение разницы пар $(Score_{Gold} - Score_{USD})$ ≤ 0.

*Альтернативная гипотеза* ***H1:***

Суть: "Золотые уровни работают иначе (лучше или хуже), и это закономерность, а не случайность".

Математически: Среднее значение разницы пар $(Score_{Gold} - Score_{USD})$ ≥ 0.

t_stat, p_value = stats.ttest_rel(gold_scores, usd_scores)

Эта функция берет два списка (оценки 7 уровней Gold и 7 уровней USD), сравнивает их попарно и вычисляет P-value


In [18]:
print("\n")
print("=" * 70)
print(f"{'АНАЛИЗ ЭФФЕКТИВНОСТИ УРОВНЕЙ':^70}")
print("=" * 70)

for item in stats_list:
    print("\n")
    print(f"{item['Name']} | Дата: {item['Date']}")
    print("=" * 70)

    # Счет метрик
    u_days = item['USD_Respect']
    g_days = item['Gold_Respect']
    u_cross = item['USD_Cross']
    g_cross = item['Gold_Cross']

    # Score
    score_u = u_days - (u_cross * 5)
    score_g = g_days - (g_cross * 5)
    best = "GOLD" if score_g > score_u else "USD"

    # Эффективность (дней на 1 пробой)
    eff_u = u_days / u_cross if u_cross > 0 else u_days
    eff_g = g_days / g_cross if g_cross > 0 else g_days

    # ТАБЛИЦА
    print(f"{'Метрика':<30} {'USD':>10} {'GOLD':>10} {'Лучше':>8}")
    print("-" * 62)
    print(f"{'Дней удержания (Respect)':<30} {u_days:>10} {g_days:>10} {'GOLD' if g_days > u_days else 'USD':>8}")
    print(f"{'Количество пробоев (Cross)':<30} {u_cross:>10} {g_cross:>10} {'GOLD' if g_cross < u_cross else 'USD':>8}")
    print(f"{'Эффективность (Days/Cross)':<30} {eff_u:>10.1f} {eff_g:>10.1f} {'GOLD' if eff_g > eff_u else 'USD':>8}")
    print("-" * 62)
    print(f"{'КАЧЕСТВО (SCORE)':<30} {score_u:>10} {score_g:>10} {best:>8}")


print("\n\n")
print("=" * 70)
print(f"{'ФИНАЛЬНЫЙ ТЕСТ ЗНАЧИМОСТИ':^70}")
print("=" * 70)

gold_scores = [x['Gold_Respect'] - (x['Gold_Cross']*5) for x in stats_list]
usd_scores = [x['USD_Respect'] - (x['USD_Cross']*5) for x in stats_list]

# Paired T-Test (Сравнение средних в зависимых выборках)
t_stat, p_value = stats.ttest_rel(gold_scores, usd_scores)

# Общая статистика
avg_gain = (np.mean(gold_scores) - np.mean(usd_scores))
gain_pct = (np.mean(gold_scores) / np.mean(usd_scores) - 1) * 100

print(f"Средний Score (GOLD): {np.mean(gold_scores):.1f}")
print(f"Средний Score (USD):  {np.mean(usd_scores):.1f}")
print(f"Средний прирост:      +{gain_pct:.1f}%")
print("-" * 70)

print(f"Paired T-test statistic: {t_stat:.3f}")
print(f"P-VALUE:                 {p_value:.5f}")

print("-" * 70)
if p_value < 0.05:
    print("[V] РЕЗУЛЬТАТ: СТАТИСТИЧЕСКИ ЗНАЧИМО (p < 0.05)")
elif p_value < 0.1:
    print("[?] РЕЗУЛЬТАТ: ПОГРАНИЧНАЯ ЗНАЧИМОСТЬ (0.05 < p < 0.1)")
    print("    Есть тенденция, но нужно больше данных.")
else:
    print("[X] РЕЗУЛЬТАТ: НЕ ЗНАЧИМО (p > 0.1)")
    print("    Разница может быть случайной.")
print("=" * 70)



                     АНАЛИЗ ЭФФЕКТИВНОСТИ УРОВНЕЙ                     


L1 (73g) | Дата: 2017-06-11
Метрика                               USD       GOLD    Лучше
--------------------------------------------------------------
Дней удержания (Respect)                3          5     GOLD
Количество пробоев (Cross)              1          1      USD
Эффективность (Days/Cross)            3.0        5.0     GOLD
--------------------------------------------------------------
КАЧЕСТВО (SCORE)                       -2          0     GOLD


L2 (475g) | Дата: 2017-12-18
Метрика                               USD       GOLD    Лучше
--------------------------------------------------------------
Дней удержания (Respect)               81        108     GOLD
Количество пробоев (Cross)              3          5      USD
Эффективность (Days/Cross)           27.0       21.6      USD
--------------------------------------------------------------
КАЧЕСТВО (SCORE)                       66         83    